In [1]:
import networkx as nx
import pickle
import random
import os

<br>
Out of the 4300~ sentences from lrec_ebm.csv
<br>
176 failed.
<br>

These 3 files from the 176 need manual intervention
<br>

344034.p
<br>
394434.p
<br>
420219.p

In [2]:
# just change the filenames

graphml_file = '344034.graphml'
pickle_file = '344034.p'

In [3]:

BASE_DIR = os.getcwd()
PICKLE_DCS_FILE_NAME = os.path.join(BASE_DIR,'data','dcs_miss',pickle_file)
GRAPHML_FILE_NAME = os.path.join(BASE_DIR,'data','dcs_miss',graphml_file)

In [4]:
# defining class for pickle file 

class DCS:
    def __init__(self, sent_id, sentence):
        self.sent_id = sent_id
        self.sentence = sentence
        self.dcs_chunks = []
        self.lemmas = []
        self.cng = []

dcs_class_params = ['sentence id', 'sentence ', 'chunks', 'lemmas', 'morph class']        

In [5]:

# Double characters mapping to single characters
_dbl = dict({
    'ai' : 'E',
    'au' : 'O',
    'kh' : 'K',
    'gh' : 'G',
    'ch' : 'C',
    'jh' : 'J',
    'ṭh' : 'W',
    'ḍh' : 'Q',
    'th' : 'T',
    'dh' : 'D',
    'ph' : 'P',
    'bh' : 'B'})

# One to one mapping
_oth = dict({
    'ā' : 'A',
    'ī' : 'I',
    'ū' : 'U',
    'ṛ' : 'f',
    'ṝ' : 'F',
    'ḷ' : 'x',
    'ḹ' : 'X',
    'ṃ' : 'M',
    'ḥ' : 'H',
    'ṁ' : '~',
    'ṅ' : 'N',
    'ñ' : 'Y',
    'ṭ' : 'w',
    'ḍ' : 'q',
    'ṇ' : 'R',
    'ś' : 'S',
    'ṣ' : 'z'})


In [6]:
##IAST to SLP1 transliteration


def iast2slp(src):
    '''
    Converts International Alphabet for Sanskrit Transliteration (IAST) scheme to
    Sanskrit Library Phonetic Basic notation
    '''
    tgt = ''
    inc = 0
    while inc < len(src):
        now = src[inc]
        nxt = src[inc+1] if inc < len(src) - 1 else ''
        if now + nxt in _dbl:
            tgt += _dbl[now + nxt]
            inc += 1
        elif now in _oth:
            tgt += _oth[now]
        else:
            tgt += now
        inc += 1
    return tgt




In [7]:
# Read the pickle file containing ground truth data





def pickle_data_give(filename):
    '''Returns the attributes of the DCS pickle file
    
    Args:
    filename (str) = name of file
    
    Returns:
    output_load.sent_id (int): sentence id
    output_load.sentence (str): sentence 
    output_load.dcs_chunks (list): chunks
    output_load.lemmas (list): lemmas
    output_load.cng (list): morphological class
    
    Raises:
    '''
    
    output_load = pickle.load(open(filename, "rb"), encoding='utf-8')
    
    
    return output_load.sent_id, output_load.sentence, output_load.dcs_chunks, output_load.lemmas, output_load.cng
    

In [8]:
# read graph ml file

graph_file = nx.read_graphml(GRAPHML_FILE_NAME)


In [9]:
# Recursively break and simplify a nested list

def simplify_nested_list(nested_list):   
    
    # requires resetting of "all_single_elements_list" variable prior to each function call
    
    if len(nested_list) >= 1 and type(nested_list) is list:
        for idx, ele in enumerate(nested_list):
            #print(idx, ele, type(ele))
            if type(ele) is str:
                
                all_single_elements_list.append([ele])
                continue
                
            else:
                simplify_nested_list(nested_list[idx])
                
        
    else:
        
        all_single_elements_list.append(nested_list)
        return (nested_list)
        


In [10]:
# algorithm to find the cng+lemma combinations

_, dcs_sentence, _, lemmas, cng = pickle_data_give(PICKLE_DCS_FILE_NAME)
#print(lemma, '\n',cng)

#print(len(dcs_sentence.strip().rstrip().split(' ')))
#print(dcs_sentence.strip().rstrip().split(' '))
graph_ground_truth_ids = []
flattened_chunk_no = []
print('==========================================================================')


#get chunk no's flattened list
for idx, lemma in enumerate(lemmas):
    #print(len(lemma))
    if len(lemma)>1:
        #print('$')
        tmp=[]
        for x in range(len(lemma)):
            tmp.append(str(idx+1))
        flattened_chunk_no.append(tmp)
    else:
        #print('%')
        flattened_chunk_no.append(str(idx+1))
        

all_single_elements_list = []
simplify_nested_list(flattened_chunk_no)
flattened_chunk_no = all_single_elements_list 
        
        
all_single_elements_list = []
simplify_nested_list(lemmas)
lemmas = all_single_elements_list


all_single_elements_list = []
simplify_nested_list(cng)
cng = all_single_elements_list

print('\n\nlemmas:\t%s \ncngs:\t%s' %(lemmas, cng))

lemma_cng = zip(lemmas, cng)

number_of_chunks = len(dcs_sentence.strip().rstrip().split(' '))


for dcs_lemma, dcs_cng in lemma_cng:
    print('\n\n******************************')
    print('\n\nCurrent lemma: %s \tCurrent cng:%s' %(dcs_lemma, dcs_cng))
    
    same_ground_truth_ids = []
    
    for di in graph_file.nodes():
        
        graph_file.add_node(str(di), ground_truth_id=0)
        
        # this is random one time traversal search in the whole graph
        #print('Current node:%s' %(di))
        temp_dict = graph_file.nodes[di]
        
        # convert the lemma to slp1
        dcs_lemma[0] = iast2slp(dcs_lemma[0])
        
        
        
        print('checking node %s' %(di))
        print('\t\tLemma:%s\tCng:%s\n' %(temp_dict['lemma'], temp_dict['cng']))
        
        # check if in the current node, the cng matches to the pickle file cng
        if temp_dict['cng'] == int(dcs_cng[0]):
                       
            
            
            # check if in the current node, the lemma matches to the pickle file lemma
            if (temp_dict['lemma'] == dcs_lemma[0]):
                
                # only if cng and the converted lemma match, then these are executed
                
                #if tempdict chunk number matches dcs chnk number
                print('\t\t\t\t\t|$| - Here in node %s' %(di))
                #graph_file.add_node(di, ground_truth_id=1)
                same_ground_truth_ids.append(di)
        else:
            
            graph_file.add_node(str(di), ground_truth_id=0)
        
    graph_ground_truth_ids.append(same_ground_truth_ids)
                
                #print('#', di, '\n')
    

#return len(graph_ground_truth_ids)/number_of_chunks



lemmas:	[['bahu'], ['yuj']] 
cngs:	[['101'], ['-190']]


******************************


Current lemma: ['bahu'] 	Current cng:['101']
checking node 21
		Lemma:ad	Cng:30

checking node 19
		Lemma:ad	Cng:71

checking node 28
		Lemma:varjita	Cng:30

checking node 27
		Lemma:ala	Cng:3

checking node 22
		Lemma:na	Cng:2

checking node 11
		Lemma:yukta	Cng:169

checking node 17
		Lemma:anna	Cng:3

checking node 4
		Lemma:yukta	Cng:29

checking node 10
		Lemma:yukta	Cng:171

checking node 2
		Lemma:bahu	Cng:101

					|$| - Here in node 2
checking node 3
		Lemma:bahu	Cng:100

checking node 13
		Lemma:eka	Cng:30

checking node 29
		Lemma:vfj	Cng:-190

checking node 24
		Lemma:viMSati	Cng:170

checking node 33
		Lemma:i	Cng:-42

checking node 5
		Lemma:yuj	Cng:-190

checking node 26
		Lemma:mala	Cng:3

checking node 20
		Lemma:ad	Cng:31

checking node 12
		Lemma:yuj	Cng:-190

checking node 30
		Lemma:varjita	Cng:101

checking node 25
		Lemma:a	Cng:3

checking node 15
		Lemma:eka	Cng:79

check

In [11]:
# this function is used to pick one cng+lemma combination if redundant units are present.

def pick_one_cng_lemma(graph_ground_truth_ids):
    
    update_ground_truth_ids = []
    graph_ground_truth_ids = list(filter(None, graph_ground_truth_ids))
    for ids_list in graph_ground_truth_ids:
        if len(ids_list) > 1:
            chose_cng_lemma = random.choice(ids_list)
            graph_file.add_node(chose_cng_lemma, ground_truth_id=1)
        else:
            chose_cng_lemma = ids_list[0]            
            graph_file.add_node(chose_cng_lemma, ground_truth_id=1)
            
        update_ground_truth_ids.append(chose_cng_lemma)
    
    return update_ground_truth_ids
    
graph_ground_truth_ids = pick_one_cng_lemma(graph_ground_truth_ids)
#print(graph_ground_truth_ids)

In [12]:
# snippet to get the ground truth sentence from the graphML file

print('Graph ground truth nodes:')
graph_ground_truth_ids = sorted(graph_ground_truth_ids, key = lambda x : graph_file.nodes[x]['chunk_no'])
print(graph_ground_truth_ids)


for ids in graph_ground_truth_ids:
    ids = str(ids)
    print(graph_file.nodes[str(ids)]['word'], end=' ')

  
print('\n\n')
print('DCS sentence below:\n')

for idx, _ in enumerate(pickle_data_give(PICKLE_DCS_FILE_NAME)):
    print('%s: \t%s' %(dcs_class_params[idx], _))

Graph ground truth nodes:
['2', '12']
bahuBiH yukte 


DCS sentence below:

sentence id: 	344034
sentence : 	bahuBiryukta ekAnnaviMSatyAmalavarjitEH    
chunks: 	['bahu', 'yuj']
lemmas: 	[['bahu', 'yuj']]
morph class: 	[['101', '-190']]


### ***The issue in the 3 dcs pickle files is that they dont have all lemma+cng information.***